In [1]:
import time 
import requests
import cv2
import operator
import numpy as np
import json

# Import compatibility libraries (python 2/3 support)
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

# Python 3
try:
    from urllib.request import urlopen, Request
    from urllib.parse import urlparse, urlencode
    from http.client import HTTPSConnection
# Python 2.7
except ImportError:
    from urlparse import urlparse
    from urllib import urlencode
    from urllib2 import Request, urlopen
    from httplib import HTTPSConnection

# Import library to display results
import matplotlib.pyplot as plt
%matplotlib inline 
# Display images within Jupyter

## Get some constants

In [2]:
# This could also be an external file
config_json = '''{
"subscription_key_computer_vision": "adab405efde94809a2e24dfd2b585b29"
}'''

CONFIG=json.loads(config_json)
# grab key from json in config
subscription_key = CONFIG['subscription_key_computer_vision']

In [3]:

# Variables# Varia 

# Make sure to take your Endpoint from Overview page of API in Azure Portal
_url = 'https://southcentralus.api.cognitive.microsoft.com/customvision/v2.0/Prediction/9f3f9c00-3ac8-40b8-b054-ffb5ead96443/url?iterationId=cdb77c52-a821-4327-b2ab-15ddce62c912' # change the ending to 'analyze' or 'describe'
_key = subscription_key

_maxNumRetries = 10

In [4]:
def processRequest( json, data, headers, params ):

    """
    Helper function to process the request to Project Oxford

    Parameters:
    json: Used when processing images from its URL. See API Documentation
    data: Used when processing image read from disk. See API Documentation
    headers: Used to pass the key information and the data type request
    """

    retries = 0
    result = None

    while True:
        # Use the requests library to make the POST call
        response = requests.request( 'post', 
                                    _url, 
                                    json=json, 
                                    data=data, 
                                    headers=headers, 
                                    params=params )

        if response.status_code == 429: 
            print( "Message: %s" % ( response.json()['error']['message'] ) )
            if retries <= _maxNumRetries: 
                time.sleep(1) 
                retries += 1
                continue
            else: 
                print( 'Error: failed after retrying!' )
                break
                
        # We have a successful response, but let's do some tests on response data
        elif response.status_code == 200 or response.status_code == 201:
            if 'content-length' in response.headers and \
                int(response.headers['content-length']) == 0: 
                    
                result = None
                
            elif 'content-type' in response.headers and \
                isinstance(response.headers['content-type'], str):
                    
                if 'application/json' in response.headers['content-type'].lower(): 
                    result = response.json() if response.content else None
                    
                elif 'image' in response.headers['content-type'].lower(): 
                    result = response.content
        else:
            print( "Error code: %d" % ( response.status_code ) )
            print( "Message: %s" % ( response.json() ) )

        break
        
    return result

In [9]:
def renderResultOnImage( result, img ):
    """Display the obtained results onto the input image"""

    if 'description' in result:
        descrip = result['description']['captions'][0]['text']
        print(descrip)
        cv2.putText(img, descrip, (30,70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 3)
    if 'categories' in result:
        # Put a rectangle around the image to reflect accent colors
        R = int(result['color']['accentColor'][:2],16)
        G = int(result['color']['accentColor'][2:4],16)
        B = int(result['color']['accentColor'][4:],16)
        if img is not None:
            cv2.rectangle(img,(0,0), (img.shape[1], img.shape[0]), color=(R,G,B), thickness=25)

        categoryName = sorted(result['categories'], key=lambda x: x['score'])[0]['name']
        print(categoryName)
        cv2.putText(img, categoryName, (30,70), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,0,0), 3)


## Analysis of an image retrieved via URL
<font color=green>Place an image URL here in the urlImage variable (must be a good quality, large image)</font>
Input requirements:

Supported image formats: JPEG, PNG, GIF, BMP.
Image file size must be less than 4MB.
Image dimensions should be greater than 50 x 50.

In [27]:
urlImage = 'https://http2.mlstatic.com/leon-adulto-de-peluche-y-felpa-wild-republic-D_NQ_NP_21186-MCO20204954214_112014-F.jpg'

In [28]:
from IPython.display import display, Image


# Computer Vision parameters
params = { 'visualFeatures' : 'Color,Categories'} 

headers = dict()
headers['Prediction-Key'] = _key
headers['Content-Type'] = 'application/json' 

json = { 'url': urlImage } 
data = None

result = processRequest(json, data, headers, params)

if result is not None:
    # Load the original image, fetched from the URL
    arr = np.asarray( bytearray( requests.get( urlImage ).content ), 
                     dtype=np.uint8 )
    arr = arr[...,::-1]
    
    img = cv2.imdecode(arr, cv2.IMREAD_COLOR)
    
    

    renderResultOnImage(result, img)
    
    if img is not None:
        plt.subplots(figsize=(15, 20))
        plt.imshow(img)
    else:
        img = Image(url=urlImage, embed=False)
        display(img)

In [24]:
import requests

In [25]:
requests.get()

TypeError: get() missing 1 required positional argument: 'url'

In [29]:
result

{'created': '2018-09-04T02:30:20.2503661Z',
 'id': '20515818-c3cb-424c-ab7c-06c4724a555e',
 'iteration': 'cdb77c52-a821-4327-b2ab-15ddce62c912',
 'predictions': [{'probability': 0.736275554,
   'tagId': '6ba2fed8-f76e-4f45-a9ad-5bca0d9db06b',
   'tagName': 'insulated_jackets'},
  {'probability': 0.263724416,
   'tagId': '2fdda095-59a3-49b9-9a47-b0a795c398f3',
   'tagName': 'hardshell_jacket'}],
 'project': '9f3f9c00-3ac8-40b8-b054-ffb5ead96443'}

In [30]:
headers

{'Content-Type': 'application/json',
 'Prediction-Key': 'adab405efde94809a2e24dfd2b585b29'}

In [34]:
response = requests.request( 'post', _url, json=json, 
                                    data=data, 
                                    headers=headers, 
                                    params=params )

In [35]:
response.content

b'{"id":"3dbf2262-7461-4045-abbd-4e781befd75b","project":"9f3f9c00-3ac8-40b8-b054-ffb5ead96443","iteration":"cdb77c52-a821-4327-b2ab-15ddce62c912","created":"2018-09-04T03:14:59.7064944Z","predictions":[{"probability":0.736275554,"tagId":"6ba2fed8-f76e-4f45-a9ad-5bca0d9db06b","tagName":"insulated_jackets"},{"probability":0.263724416,"tagId":"2fdda095-59a3-49b9-9a47-b0a795c398f3","tagName":"hardshell_jacket"}]}'